# Pick highest-employment duplicate
This script does the following:
1. Takes in a CSV with all records that are potential duplicate records
2. On each unique lat-long:  
    a. iterate through each record on the location  
    b. for each record on location, iterate through all at same location  
    c. if it hits a match, tag with the name of the current iteration's name
    

In [13]:
import pandas as pd
from fuzzywuzzy import fuzz

csv_in = r"\\data-svr\Monitoring\Employment Inventory\Employment 2020\CSVs with Flags\Testing\Table 3.csv"

df = pd.read_csv(csv_in)
df.head()

col_name = 'coname'
col_dflag = 'dupe_flag'
col_locid = 'latlon_uid'
col_locnum = 'locnum'
col_drefname = 'duperefname'

df[col_drefname] = '0'
    


In [15]:
loc_uids = list(df[col_locid].unique())

# print(len(loc_uids))
# print(loc_uids[0])

for i, uid in enumerate(loc_uids):
    # df[col_drefname] = df[col_name].apply(lambda x: get_duperef(df, x, uid))
#     df.loc[df[col_locid] == uid][col_drefname] = df.loc[df[col_locid] == uid][col_name] \
#                                                         .apply(lambda x: get_duperef(df.loc[df[col_locid] == uid], x, uid))
    
    dff = df.loc[df[col_locid] == uid] # select only records on the parcel
    conames_recs = dff[[col_name, col_locnum]].to_records(index=False) # convert to list of pairs
    
    cnames = [c[0] for c in conames_recs]
    
    for search_str in cnames:
        for cname in conames_recs:
            reccname = cname[0] # coname
            cid = cname[1] # locnum values

            try:
                ref_val = dff.loc[dff[col_locnum] == cid].reset_index()[col_drefname][0]
            except:
                print(df.loc[dff[col_locnum] == cid])
                import pdb; pdb.set_trace()


            if ref_val != '0':
                continue
            elif fuzz.ratio(search_str, reccname) > 80:
                # import pdb; pdb.set_trace()
                df.loc[df[col_locnum] == cid, col_drefname] = search_str
            else:
                continue

    if i % 250 == 0:
        print(f"{i} records processed.")
            
# df.to_csv(r"P:\Employment Inventory\Employment 2020\CSVs with Flags\Testing\Table3_out.csv", index=False)

0 records processed.
250 records processed.
500 records processed.
750 records processed.
1000 records processed.
1250 records processed.
1500 records processed.
1750 records processed.
2000 records processed.
2250 records processed.
2500 records processed.
2750 records processed.


In [60]:
# sorting and plucking

dfs = df.sort_values(by=[col_locid, col_drefname, 'locemp'], ascending=[True, True, False])

# get the version of the employer with the most employees at the location
dfg = dfs.groupby([col_locid, col_drefname])['locemp'].max()

# dfs.head(10)
# dfg.reset_index().head(10)

# merge: resulting locemp_y will be locemp of the version of the employer with largest number of employees
dfjn = dfs.merge(dfg, on=[col_locid, col_drefname])
dfjn.head()

dfj_dict = dfjn.to_dict(orient='index')

outlist = []

for k, v in dfj_dict.items():
    empx = v['locemp_x']
    empy = v['locemp_y']
    lnum = v['locnum']
    stdname = v[col_drefname]
    
    




# # filter so that resulting records are just those that have the most employees, and are thus presumed to be the 'real' ones
# dfjn = dfjn.loc[dfjn['locemp_x'] == dfjn['locemp_y']]

# dfjn.head(10)

# uvals = ['coname', 'staddr', 'locemp_x', 'naics4', 'latlon_uid', 'duperefname']
# dfjn = dfjn[uvals].drop_duplicates()


# dfs2 = dfs[['latlon_uid', 'duperefname', 'locnum']]
# dfjn = dfjn.merge(dfs, on=[col_locid, col_drefname])

# dfjn.to_csv(r"P:\Employment Inventory\Employment 2020\CSVs with Flags\Testing\Table3_out_biggestOnly.csv", index=False)


,coname,staddr,locnum,locemp_x,dupe_flag,naics4,latlon_uid,duperefname,locemp_y
0,Phillips Chiropractic,375 W Main St Ste D,881496715,6,DSM_only,8121,0,Phillips Chiropractic,6
1,Phillips Chiropractic,375 W Main St Ste D,427914873,3,DMN,6213,0,Phillips Chiropractic,6
2,Raley's,367 W Main St,104833801,115,DSM_only,4451,0,Raley's,115
3,Raleys,367 W Main St,707189713,1,DSM_only,6213,0,Raley's,115
4,Sacramento Locks,2546 Del Paso Blvd,405843887,3,DMN,5616,2,Sacramento Locksmith,3
